In [4]:
# 本代码由可视化策略环境自动生成 2022年7月7日 10:17
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


# Python 代码入口函数，input_1/2/3 对应三个输入端，data_1/2/3 对应三个输出端
def m1_run_bigquant_run(input_1, input_2, input_3):
    # 示例代码如下。在这里编写您的代码
    df = DataSource('47bbe07ecb7d470dbe75eeb8ba74c1a1T').read()
    df_2 = DataSource('88a1c0bf042e4a7f841063f2c64c2b54T').read()
    data_1 = DataSource.write_pickle(df)
    data_2 = DataSource.write_pickle(df_2)
    return Outputs(data_1=data_1, data_2=data_2, data_3=None)

# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m1_post_run_bigquant_run(outputs):
    return outputs

# 自定义Python模块
# 函数m2_run_bigquant_run、bigquant_postrun命名不可修改
# 测试“自定义Python模块”时，如果依赖输入，点击按钮“在上方插入代码单元”，构建数据。
import os
import sys
import time
import datetime
import empyrical
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
# 读取数据
# from bigdatasource import DataSource
# import biglearning.api.tools as T
# DIR_ROOT = os.path.dirname(sys._getframe().f_back.f_code.co_filename)
DIR_ROOT = os.getcwd()


class interactModule(object):
    '''
        data_source: 数据字典
                dict_keys([ret, chaoe_df, total, cost, holdcap, target_w, hold_w, tradeprice, hold_num, sell_num, buy_num, next_weights_data, res_style])
        benchmark_instruments: 基准标的代码
                默认"000905.SH"
    '''
    def __init__(self, data_source, data_source_2, benchmark_instruments="000905.SH"):
        # 获取数据
        ret, chaoe_df, total, cost, holdcap, target_w, hold_w, tradeprice, hold_num, sell_num, buy_num, next_weights_data, res_style = data_source["ret"], data_source["chaoe_df"], \
        data_source["total"], data_source["cost"], data_source["holdcap"], data_source["target_w"], data_source["hold_w"], data_source["tradeprice"], data_source["hold_num"], \
        data_source["sell_num"], data_source["buy_num"], data_source["next_weights_data"], data_source["res_style"]
        benchmark_return, details, statistics, orders, positions, transactions = data_source_2['benchmark_return'], data_source_2['details'], data_source_2['statistics'],\
        data_source_2['orders'], data_source_2['positions'], data_source_2['transactions']

        # 全局参数
        self.page_margin = 20
        self.positions_page_num = max(0, len(positions) - 1) // self.page_margin + 1
        self.transactions_page_num = max(0, len(transactions) - 1) // self.page_margin + 1
        self.orders_page_num = max(0, len(orders) - 1) // self.page_margin + 1

        # 数据
        self.base = benchmark_return.cumsum()
        self.dates = chaoe_df.index
        self.chaoe = chaoe_df.chaoe.fillna(0).cumsum()
        self.strategy = self.base + self.chaoe
        self.hold = holdcap.sum(1) / total
        self.returns = pd.concat([self.strategy, self.base, self.chaoe, self.hold], axis=1, ignore_index=False)
        self.returns.rename(columns={
                        0:"策略收益率",
                        "s_dq_close":"基准收益率",
                        "chaoe":"相对收益率",
                        1:"持仓占比"}, inplace=True)
        self.positions = positions.reindex(positions.index.union(range(len(positions), self.page_margin * self.positions_page_num))
                                           )[["date", "sid", "name", "amount_after_adjust", "value", "changew"]]
        self.positions.rename(columns={
                        "date": "日期时间",
                        "sid": "股票代码",
                        "name": "股票名称",
                        "amount_after_adjust": "持仓数量",
                        "value": "资金仓位",
                        "changew": "持仓权重"}, inplace=True)
        self.transactions = transactions.reindex(transactions.index.union(range(len(transactions), self.page_margin * self.transactions_page_num))
                                                 )[["date", "sid", "name", "direction", "amount", "price_after_adjust", "value", "cost"]]
        self.transactions.rename(columns={
                        "date": "日期时间",
                        "sid": "股票代码",
                        "name": "股票名称",
                        "direction": "操作",
                        "amount": "成交量",
                        "price_after_adjust": "成交价格",
                        "value": "成交额",
                        "cost": "手续费"}, inplace=True)
        self.orders = orders.reindex(orders.index.union(range(len(orders), self.page_margin * self.orders_page_num))
                                     )[["date", "sid", "name", "direction", "amount", "price", "targetw", "lastw"]]
        self.orders.rename(columns={
                        "date": "日期时间",
                        "sid": "股票代码",
                        "name": "股票名称",
                        "direction": "操作",
                        "amount": "委托数量",
                        "price": "价格",
                        "targetw": "计划权重",
                        "lastw": "上一日权重"}, inplace=True)
        self.details = details
        self.statistics = statistics
        self.res_style = res_style
        self.date_map = {self.date_format_transfer(date): i for i, date in enumerate(self.dates)}


        ###### 日期模块 ######
        # 持仓详情查询日期
        positions_date = widgets.DatePicker(description='', disabled=False,
                                                 layout=widgets.Layout(width="125px"))
        # 交易详情查询日期
        transactions_date = widgets.DatePicker(description='', disabled=False,
                                               layout=widgets.Layout(width="125px"))

        ###### 文本模块 ######
        # 策略
        strategy_label = widgets.HTML(value="<span style='font-size:20px;'> <b>策略名称</b> </span>")
        cum_return_label = widgets.HTML(
            value="<span style='font-size:30px;color:{}'> <b>{:.2f}%</b> </span>".format(
                "red" if self.details["累计收益率"].sum() > 0 else "green", self.details["累计收益率"].sum() * 100))
        cum_text_label = widgets.HTML(value="累计收益")
        relative_cum_return_label = widgets.HTML(
            value="<span style='font-size:30px;color:{}'> <b>{:.2f}%</b> </span>".format(
                "red" if self.details["超额累计收益率"].sum() > 0 else "green", self.details["超额累计收益率"].sum() * 100))
        relative_cum_text_label = widgets.HTML(value="超额累计收益")
        today_return_label = widgets.HTML(
            value="今日收益<span style='color:{}'> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>{:.2f}%</b> </span>".format(
                "red" if self.details["今日收益率"].sum() > 0 else "green", self.details["今日收益率"].sum() * 100))
        annual_return_label = widgets.HTML(
            value="年化收益<span style='color:{}'> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>{:.2f}%</b> </span>".format(
                "red" if self.details["年化收益率"].sum() > 0 else "green", self.details["年化收益率"].sum() * 100))
        max_draw_label = widgets.HTML(
            value="最大回撤&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>{:.2f}%</b>".format(self.details["最大回撤"].sum()))
        sharpe_ratio_label = widgets.HTML(
            value="夏普比率&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>{:.2f}</b>".format(self.details["夏普比率"].sum()))
        init_cash_label = widgets.HTML(
            value="初始资金&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>{:.0f}</b>".format(self.details["初始资金"].sum()))
        trade_start_label = widgets.HTML(
            value="开始时间&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>{}</b>".format(self.details["开始时间"].sum()))

        # 收益率走势
        plot_label = widgets.HTML(value="<span style='font-size:20px;'> <b>收益率走势</b> </span>")
        # 收益统计
        statistic_label = widgets.HTML(value="<span style='font-size:20px;'> <b>收益统计</b> </span>")
        # 风格偏好
        styles_label = widgets.HTML(value="<span style='font-size:20px;'> <b>风格偏好</b> </span>")
        # 计划交易
        orders_label = widgets.HTML(value="<span style='font-size:20px;'> <b>计划交易</b> </span>")
        orders_page_label = widgets.HTML(value="/%s页" % str(self.orders_page_num))
        # 持仓详情
        positions_label = widgets.HTML(value="<span style='font-size:20px;'> <b>持仓详情</b> </span>")
        self.positions_page_label = widgets.HTML(value="/%s页" % str(self.positions_page_num))
        # 交易详情
        transactions_label = widgets.HTML(value="<span style='font-size:20px;'> <b>交易详情</b> </span>")
        self.transactions_page_label = widgets.HTML(value="/%s页" % str(self.transactions_page_num))
        

        ###### 按键模块 ######
        # 计划交易下载按键
        orders_download_button = widgets.Button(
            description='下载',
            disabled=False,
            button_style='',
            tooltip='下载计划交易表格',
            icon='download',
            layout=widgets.Layout(width="70px")
        )
        # 计划交易翻页按键
        orders_previous_page_button = widgets.Button(
            description='',
            disabled=False,
            button_style='',
            tooltip='上一页',
            icon='angle-left',
            layout=widgets.Layout(width="30px")
        )
        self.orders_cur_page = widgets.BoundedIntText(
            value=1,
            min=1,
            max=self.orders_page_num,
            step=1,
            description='',
            disabled=False,
            layout=widgets.Layout(width="60px")
        )
        orders_next_page_button = widgets.Button(
            description='',
            disabled=False,
            button_style='',
            tooltip='下一页',
            icon='angle-right',
            layout=widgets.Layout(width="30px")
        )
        # 持仓详情下载按键
        positions_download_button = widgets.Button(
            description='下载',
            disabled=False,
            button_style='',
            tooltip='下载持仓详情表格',
            icon='download',
            layout=widgets.Layout(width="70px")
        )
        # 持仓详情翻页按键
        positions_previous_page_button = widgets.Button(
            description='',
            disabled=False,
            button_style='',
            tooltip='上一页',
            icon='angle-left',
            layout=widgets.Layout(width="30px")
        )
        self.positions_cur_page = widgets.BoundedIntText(
            value=1,
            min=1,
            max=self.positions_page_num,
            step=1,
            description='',
            disabled=False,
            layout=widgets.Layout(width="60px")
        )
        positions_next_page_button = widgets.Button(
            description='',
            disabled=False,
            button_style='',
            tooltip='下一页',
            icon='angle-right',
            layout=widgets.Layout(width="30px")
        )

        # 交易详情下载按键
        transactions_download_button = widgets.Button(
            description='下载',
            disabled=False,
            button_style='',
            tooltip='下载交易详情表格',
            icon='download',
            layout=widgets.Layout(width="70px")
        )
        # 交易详情翻页按键
        transactions_previous_page_button = widgets.Button(
            description='',
            disabled=False,
            button_style='',
            tooltip='上一页',
            icon='angle-left',
            layout=widgets.Layout(width="30px")
        )
        self.transactions_cur_page = widgets.BoundedIntText(
            value=1,
            min=1,
            max=self.transactions_page_num,
            step=1,
            description='',
            disabled=False,
            layout=widgets.Layout(width="60px")
        )
        transactions_next_page_button = widgets.Button(
            description='',
            disabled=False,
            button_style='',
            tooltip='下一页',
            icon='angle-right',
            layout=widgets.Layout(width="30px")
        )

        # 翻页交互
        orders_previous_page_button.on_click(self.orders_previous_page_turn)
        orders_next_page_button.on_click(self.orders_next_page_turn)
        positions_previous_page_button.on_click(self.positions_previous_page_turn)
        positions_next_page_button.on_click(self.positions_next_page_turn)
        transactions_previous_page_button.on_click(self.transactions_previous_page_turn)
        transactions_next_page_button.on_click(self.transactions_next_page_turn)

        # 交互更新
        positions_date.observe(self.update_positions_page_num, "value")
        transactions_date.observe(self.update_transactions_page_num, "value")
        
        # 绘图交互
        interactive_plot = widgets.interactive(self.display_figure)
        
        # 绘图布局
        self.plot_grid = widgets.GridspecLayout(2, 1)
        self.plot_grid[0, 0] = plot_label
        self.plot_grid[1, 0] = interactive_plot
        self.plot_grid.layout.grid_template_rows = "60px auto"
        self.plot_grid.layout.grid_template_columns = 'auto'
        self.plot_grid.layout.align_items = 'center'
        # self.plot_grid.layout.justify_content = "space-between"
        # self.plot_grid.layout.justify_items = 'center'

        # 表格交互
        positions_download_button.on_click(self.save_positions)
        positions_interactive_table = widgets.interactive_output(self.output_positions_table, {
                                                                "date": positions_date,
                                                                "page": self.positions_cur_page
                                                            })

        # 持仓详情翻页布局
        positions_page_grid = widgets.GridspecLayout(1, 5)
        positions_page_grid[0, 3] = self.positions_page_label
        positions_page_grid[0, 1] = positions_previous_page_button
        positions_page_grid[0, 2] = self.positions_cur_page
        positions_page_grid[0, 4] = positions_next_page_button
        positions_page_grid.layout.grid_template_rows = "40px"
        positions_page_grid.layout.grid_template_columns = "auto 40px 70px 60px 40px"

        self.positions_grid = widgets.GridspecLayout(3, 4)
        self.positions_grid[0, 0] = positions_label
        self.positions_grid[0, 1] = positions_date
        self.positions_grid[0, 3] = positions_download_button
        self.positions_grid[1, :] = positions_interactive_table
        self.positions_grid[2, :] = positions_page_grid
        self.positions_grid.layout.grid_template_rows = "60px auto 40px"
        self.positions_grid.layout.grid_template_columns = "100px 130px auto 80px"
        self.positions_grid.layout.align_items = 'center'
        # self.positions_grid.layout.justify_content = "space-between"
        # self.positions_grid.layout.justify_items = 'center'
        
        # 表格交互
        transactions_download_button.on_click(self.save_transactions)
        transactions_interactive_table = widgets.interactive_output(self.output_transactions_table, {
                                                                "date": transactions_date,
                                                                "page": self.transactions_cur_page
                                                            })
        # 交易详情翻页布局
        transactions_page_grid = widgets.GridspecLayout(1, 5)
        transactions_page_grid[0, 3] = self.transactions_page_label
        transactions_page_grid[0, 1] = transactions_previous_page_button
        transactions_page_grid[0, 2] = self.transactions_cur_page
        transactions_page_grid[0, 4] = transactions_next_page_button
        transactions_page_grid.layout.grid_template_rows = "40px"
        transactions_page_grid.layout.grid_template_columns = "auto 40px 70px 60px 40px"

        self.transactions_grid = widgets.GridspecLayout(3, 4)
        self.transactions_grid[0, 0] = transactions_label
        self.transactions_grid[0, 1] = transactions_date
        self.transactions_grid[0, 3] = transactions_download_button
        self.transactions_grid[1, :] = transactions_interactive_table
        self.transactions_grid[2, :] = transactions_page_grid
        self.transactions_grid.layout.grid_template_rows = "60px auto 40px"
        self.transactions_grid.layout.grid_template_columns = "100px 130px auto 80px"
        self.transactions_grid.layout.align_items = 'center'
        # self.transactions_grid.layout.justify_content = "space-between"
        # self.transactions_grid.layout.justify_items = 'center'

        # 策略详情
        detail_grid = widgets.GridspecLayout(3, 4)
        detail_grid[0:2, 0] = cum_return_label
        detail_grid[2, 0] = cum_text_label
        detail_grid[0:2, 1] = relative_cum_return_label
        detail_grid[2, 1] = relative_cum_text_label
        detail_grid[0, 2] = today_return_label
        detail_grid[1, 2] = max_draw_label
        detail_grid[2, 2] = init_cash_label
        detail_grid[0, 3] = annual_return_label
        detail_grid[1, 3] = sharpe_ratio_label
        detail_grid[2, 3] = trade_start_label
        detail_grid.layout.grid_template_rows = "auto auto auto"
        detail_grid.layout.grid_template_columns = "auto auto auto auto"
        detail_grid.layout.align_items = 'center'
        detail_grid.layout.justify_items = 'center'

        self.strategy_grid = widgets.GridspecLayout(2, 1)
        self.strategy_grid[0, 0] = strategy_label
        self.strategy_grid[1, 0] = detail_grid
        self.strategy_grid.layout.grid_template_rows = "60px auto"
        self.strategy_grid.layout.grid_template_columns = "auto"
        self.strategy_grid.layout.align_items = 'center'
        # self.strategy_grid.layout.justify_content = "space-between"
        # self.strategy_grid.layout.justify_items = 'center'

        # 收益统计
        statistic_interactive_table = widgets.interactive(self.output_statistics_table)
        self.statistic_grid = widgets.GridspecLayout(2, 1)
        self.statistic_grid[0, 0] = statistic_label
        self.statistic_grid[1, 0] = statistic_interactive_table
        self.statistic_grid.layout.grid_template_rows = "60px auto"
        self.statistic_grid.layout.grid_template_columns = "auto"
        self.statistic_grid.layout.align_items = 'center'
        # self.statistic_grid.layout.justify_content = "space-between"
        # self.statistic_grid.layout.justify_items = 'center'

        # 风格因子
        styles_interactive_table = widgets.interactive(self.output_styles_table)
        self.styles_grid = widgets.GridspecLayout(2, 1)
        self.styles_grid[0, 0] = styles_label
        self.styles_grid[1, 0] = styles_interactive_table
        self.styles_grid.layout.grid_template_rows = "60px auto"
        self.styles_grid.layout.grid_template_columns = "auto"
        self.styles_grid.layout.align_items = 'center'
        # self.styles_grid.layout.justify_content = "space-between"
        # self.styles_grid.layout.justify_items = 'center'

        # 表格交互
        orders_download_button.on_click(self.save_orders)
        orders_interactive_table = widgets.interactive_output(self.output_orders_table, {"page": self.orders_cur_page})

        # 计划交易翻页布局
        orders_page_grid = widgets.GridspecLayout(1, 5)
        orders_page_grid[0, 3] = orders_page_label
        orders_page_grid[0, 1] = orders_previous_page_button
        orders_page_grid[0, 2] = self.orders_cur_page
        orders_page_grid[0, 4] = orders_next_page_button
        orders_page_grid.layout.grid_template_rows = "40px"
        orders_page_grid.layout.grid_template_columns = "auto 40px 70px 60px 40px"

        # 计划交易
        self.orders_grid = widgets.GridspecLayout(3, 4)
        self.orders_grid[0, 0] = orders_label
        self.orders_grid[0, 3] = orders_download_button
        self.orders_grid[1, :] = orders_interactive_table
        self.orders_grid[2, :] = orders_page_grid
        self.orders_grid.layout.grid_template_rows = "60px auto 40px"
        self.orders_grid.layout.grid_template_columns = "100px 130px auto 80px"
        self.orders_grid.layout.align_items = 'center'
        # self.orders_grid.layout.justify_content = "space-between"
        # self.orders_grid.layout.justify_items = 'center'

        self.total_page_grid = widgets.GridspecLayout(7, 1)
        self.total_page_grid[0, 0] = self.strategy_grid
        self.total_page_grid[1, 0] = self.plot_grid
        self.total_page_grid[2, 0] = self.statistic_grid
        self.total_page_grid[3, 0] = self.styles_grid
        self.total_page_grid[4, 0] = self.orders_grid
        self.total_page_grid[5, 0] = self.positions_grid
        self.total_page_grid[6, 0] = self.transactions_grid
        self.total_page_grid.layout.grid_template_rows = "auto"
        self.total_page_grid.layout.grid_template_columns = "auto"
        self.total_page_grid.layout.align_items = 'center'
        # self.total_page_grid.layout.justify_content = "space-between"
        # self.total_page_grid.layout.justify_items = 'center'
#         display(self.total_page_grid)
    
    def date_format_transfer(self, date):
        time_tuple = time.strptime(date, '%Y%m%d')
        year, month, day = time_tuple[:3]
        return datetime.date(year, month, day)
    

    def display_figure(self):
        self.returns["date"] = pd.to_datetime(self.returns.index.map(lambda x:self.date_format_transfer(x)))
        self.returns.set_index("date", inplace=True)
        self.returns.index.name = None
        options = {
            "legend": {
                "enabled": True,
                "symbolHeight": 8,
                "symbolWidth": 8,
                "symbolPadding": 8,
                "symbolRadius": 0,
            },
            "plotOptions": {
                "series": {
                    "showInNavigator": True,
                },
                "area": {
                    "showInNavigator": True,
                },
            },
            "rangeSelector": {
                "enabled": True,
            },
            "navigator": {
                "enabled": True,
            },
            "series": [
                {
                    "name": "策略收益率",
                    "color": "#ff0000",
                    "tooltip": {
                        "valueDecimals": 2,
                        "valueSuffix": " %",
                    },
                    "type": "area",
                    "fillColor": {
                        "linearGradient": [0, 0, 0, 1],
                        "stops": [
                            [0, "rgba(238,149,148,0.5)"],
                            [1, "rgba(238,149,148,0.5)"],
                        ],
                    },
                },
                {
                    "name": "基准收益率",
                    "color": "#2776B6",
                    "tooltip": {
                        "valueDecimals": 2,
                        "valueSuffix": " %",
                    },
                },
                {
                    "name": "相对收益率",
                    "color": "#8f4e4f",
                    "tooltip": {
                        "valueDecimals": 2,
                        "valueSuffix": " %",
                    },
                    "visible": False,
                },
                {
                    "name": "持仓占比",
                    "color": "#66ff99",
                    "tooltip": {
                        "valueDecimals": 2,
                        "valueSuffix": " %",
                    },
                },
            ],
            "yAxis": [
                {
                    "title": {
                        "text": "净值曲线",
                    },
                    "labels": {"format": "{value}%"},
                    "gridLineColor": "#cfcfcf",
                    "gridLineWidth": 1,
                },
                {
                    "title": {
                        "text": "持仓占比",
                    },
                    "labels": {"format": "{value}%"},
                    "gridLineColor": "#cfcfcf",
                    "gridLineWidth": 1,
                },
            ],
            "chart": {
                # 'height': 500,
            },
        }
        #T.plot(self.returns*100, panes=[["策略收益率", "基准收益率", "相对收益率", "80%"], ["持仓占比", "20%"]], options=options)
        

    ###### 按键响应函数 #####
    def save_figure(self, sender):
        file_name = '收益率曲线_' + datetime.datetime.now().strftime("%Y%m%d%H%M%S") + '.png'
        self.fig.savefig(os.path.join(DIR_ROOT, file_name))
        print('%s!' % sender.tooltip)


    def save_orders(self, sender):
        file_name = '计划交易_' + datetime.datetime.now().strftime("%Y%m%d%H%M%S") + '.csv'
        self.orders.to_csv(os.path.join(DIR_ROOT, file_name), encoding="utf-8", header=True, index=False)
        print('%s!' % sender.tooltip)


    def save_positions(self, sender):
        file_name = '持仓详情_' + datetime.datetime.now().strftime("%Y%m%d%H%M%S") + '.csv'
        self.positions.to_csv(os.path.join(DIR_ROOT, file_name), encoding="utf-8", header=True, index=False)
        print('%s!' % sender.tooltip)

    def save_transactions(self, sender):
        file_name = '交易详情_' + datetime.datetime.now().strftime("%Y%m%d%H%M%S") + '.csv'
        self.transactions.to_csv(os.path.join(DIR_ROOT, file_name), encoding="utf-8", header=True, index=False)
        print('%s!' % sender.tooltip)


    def orders_previous_page_turn(self, sender):
        self.orders_cur_page.value -= 1
        # print('%s!' % sender.tooltip)


    def orders_next_page_turn(self, sender):
        self.orders_cur_page.value += 1
        # print('%s!' % sender.tooltip)


    def positions_previous_page_turn(self, sender):
        self.positions_cur_page.value -= 1
        # print('%s!' % sender.tooltip)


    def positions_next_page_turn(self, sender):
        self.positions_cur_page.value += 1
        # print('%s!' % sender.tooltip)


    def transactions_previous_page_turn(self, sender):
        self.transactions_cur_page.value -= 1
        # print('%s!' % sender.tooltip)


    def transactions_next_page_turn(self, sender):
        self.transactions_cur_page.value += 1
        # print('%s!' % sender.tooltip)


    def output_positions_table(self, date, page):
        positions_style = {
            "资金仓位": "{:.2f}",
            "持仓数量": "{:.0f}"
        }
        if not date:
            display(self.positions[self.page_margin*(page-1):self.page_margin*page].style.format(None, na_rep="-").format(positions_style, na_rep="-").hide_index().set_properties(**{'width': '500px'}))
        elif date not in self.date_map:
            display("暂无数据")
        else:
            chosen_date = date.strftime("%Y-%m-%d")
            tmp = self.positions.query("日期时间 == @chosen_date").reset_index(drop=True)
            tmp = tmp.reindex(tmp.index.union(range(len(tmp), self.page_margin * (max(0, tmp.shape[0] - 1) // self.page_margin + 1))))
            display(tmp[self.page_margin*(page-1):self.page_margin*page].style.format(None, na_rep="-").format(positions_style, na_rep="-").hide_index().set_properties(**{'width': '500px'}))


    def output_transactions_table(self, date, page):
        transactions_style = {
            "成交量": "{:.0f}",
            "成交价格": "{:.2f}",
            "手续费": "{:.2f}",
            "成交额": "{:.2f}"
        }
        if not date:
            display(self.transactions[self.page_margin*(page-1):self.page_margin*page].style.format(None, na_rep="-").format(transactions_style, na_rep="-").hide_index().
                    applymap(self.highlight_cols, subset=pd.IndexSlice[:, ['操作', '成交额']]).set_properties(**{'width': '500px'}))
        elif date not in self.date_map:
            display("暂无数据")
        else:
            chosen_date = date.strftime("%Y-%m-%d")
            tmp = self.transactions.query("日期时间 == @chosen_date").reset_index(drop=True)
            tmp = tmp.reindex(tmp.index.union(range(len(tmp), self.page_margin * (max(0, tmp.shape[0] - 1) // self.page_margin + 1))))
            display(tmp[self.page_margin*(page-1):self.page_margin*page].style.format(None, na_rep="-").format(transactions_style, na_rep="-").hide_index().
                    applymap(self.highlight_cols, subset=pd.IndexSlice[:, ['操作', '成交额']]).set_properties(**{'width': '500px'}))


    def output_orders_table(self, page):
        orders_style = {
            "价格": "{:.2f}",
            "委托数量": "{:.0f}",
        }
        display(self.orders[self.page_margin*(page-1):self.page_margin*page].style.format(None, na_rep="-").format(orders_style, na_rep="-").hide_index().
                applymap(self.highlight_cols, subset=pd.IndexSlice[:, ['操作']]).set_properties(**{'width': '500px'}))


    def output_statistics_table(self):
        statistics_style = {
            "累计收益率": "{:.2%}",
            "年化收益率": "{:.2%}",
            "最大回撤": "{:.2%}",
            "胜率": "{:.2%}",
            "夏普比率": "{:.2f}",
            "今日收益率": "{:.2%}",
            "近一周收益率": "{:.2%}",
            "近一月收益率": "{:.2%}",
            "近三月收益率": "{:.2%}",
            "近半年收益率": "{:.2%}",
            "近一年收益率": "{:.2%}"
        }
        display(self.statistics.style.format(statistics_style, na_rep="-").hide_index().
                applymap(self.highlight_cols, subset=pd.IndexSlice[:, ["累计收益率", "年化收益率", "今日收益率", "近一周收益率",
                "近一月收益率", "近三月收益率", "近半年收益率", "近一年收益率"]]).set_properties(**{'width': '500px'}))


    def output_styles_table(self):
        # 使用seaborn获取颜色
#         import seaborn as sns
#         cmap = sns.light_palette("red", as_cmap=True)
        display(self.res_style.style.format("{:.2f}", na_rep="-").background_gradient(axis=1, cmap="coolwarm").
                set_properties(**{'width': '500px'}))


    def highlight_cols(self, column):
        color = ""
        if isinstance(column, str):
            if column == "卖":
                color = 'green'
            elif column == "买":
                color = 'red'
        if isinstance(column, float):
            if column < 0:
                color = 'green'
            elif column > 0:
                color = 'red'
        return 'color: %s' % color


    def update_positions_page_num(self, change):
        if change.new:
            chosen_date = change.new.strftime("%Y-%m-%d")
            tmp = self.positions.query("日期时间 == @chosen_date")
            self.positions_page_label.value = "/%s页" % str(max(0, tmp.shape[0] - 1) // self.page_margin + 1)
            self.positions_cur_page.value = 1
            self.positions_cur_page.max = max(0, tmp.shape[0] - 1) // self.page_margin + 1


    def update_transactions_page_num(self, change):
        if change.new:
            chosen_date = change.new.strftime("%Y-%m-%d")
            tmp = self.transactions.query("日期时间 == @chosen_date")
            self.transactions_page_label.value = "/%s页" % str(max(0, tmp.shape[0] - 1) // self.page_margin + 1)
            self.transactions_cur_page.value = 1
            self.transactions_cur_page.max = max(0, tmp.shape[0] - 1) // self.page_margin + 1


# Python 代码入口函数，input_1/2/3 对应三个输入端，data_1/2/3 对应三个输出端
def m2_run_bigquant_run(input_1, input_2, benchmark_instruments):
    # 示例代码如下。在这里编写您的代码
    im = interactModule(input_1.read(), input_2.read(), benchmark_instruments[0])
    display(im.total_page_grid)
    return Outputs()
# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m2_post_run_bigquant_run(outputs):
    return outputs


m1 = M.cached.v3(
    run=m1_run_bigquant_run,
    post_run=m1_post_run_bigquant_run,
    input_ports='',
    params='{}',
    output_ports=''
)

m2 = M.cached.v3(
    input_1=m1.data_1,
    input_2=m1.data_2,
    run=m2_run_bigquant_run,
    post_run=m2_post_run_bigquant_run,
    input_ports='input_1, input_2',
    params="""{
    'benchmark_instruments': '000905.SH'
}""",
    output_ports='',
    m_cached=False
)


[2022-07-06 19:10:06.235740] INFO: moduleinvoker: cached.v3 开始运行..

[2022-07-06 19:10:06.251952] INFO: moduleinvoker: 命中缓存

[2022-07-06 19:10:06.254770] INFO: moduleinvoker: cached.v3 运行完成[0.019024s].

[2022-07-06 19:10:06.280118] INFO: moduleinvoker: cached.v3 开始运行..

GridspecLayout(children=(GridspecLayout(children=(HTML(value="<span style='font-size:20px;'> <b>策略名称</b> </spa…

[2022-07-06 19:10:08.823791] INFO: moduleinvoker: cached.v3 运行完成[2.543635s].